<a href="https://colab.research.google.com/github/ZijianDu/C-in-C-KPI/blob/main/KPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import sklearn